In [1]:
from collections import defaultdict  
import pandas as pd
import numpy as np
import tools as t
import scipy.stats
import math
from collections import Counter
import pdb

In [2]:
def sub_vec( x,y ):
    return [  y[i] - x[i]   for i in xrange(len(x)) ]
def calulate_entropies_numerical(  row , N  ):
    row_sum = sum(row)
    entropy = 0
    for i,v in enumerate(row):
        if v > 0:
            p = float( v ) / row_sum
            entropy+= p*math.log(p,2)
    
    p_subset = float( row_sum) /N
    entropy =  p_subset * abs( entropy)
    return entropy

In [3]:
# calculating entropies
def calulate_entropies_categorical( splits, table,n ,unique_classes):
    #print "Evaluating Split: ",splits
    #print "Unique_classes", unique_classes
    #print table
    entropy = 0
    chota_n = 0
    
    for aik_class in unique_classes:
        numerator = 0
        denominator = 0
        
        if isinstance( splits, str ):
            #print  "len of splits", len(splits)," splits" , splits
            numerator+= table[ splits ][aik_class]
            denominator+= sum( table[splits  ].values())
            #print "Numerator" ,numerator  ,"Denum ", denominator
        else:
            for i in xrange( len(splits)):
                #print "Claaass ", aik_class ," Split Value ", splits[i]
                numerator+= table[ splits[i] ][aik_class]
                denominator+= sum( table[splits[i]  ].values())
                #print "Numerator" ,numerator  ,"Denum ", denominator
            
        #print "num ", numerator , " den" , denominator
        chota_n = denominator
        
        if numerator > 0:
            p = float(numerator ) / denominator
            entropy += p * math.log(p,2)
    
    #print "simple entropy" , entropy
    #print "chota_n" , chota_n , "bara_n" , n , "abs_entropy_withput_p", abs(  entropy )
    subset_p = float(chota_n) / n
    #print "entropy ",  subset_p * abs(  entropy )     
    
    return subset_p * abs(  entropy )   

In [4]:
class Node:
    def __init__(self,purity,klasslabel='',score=0,split=[],fidx=-1):
        self.lchild=None       
        self.rchild=None
        self.klasslabel=klasslabel        
        self.split=split
        self.score=score
        self.fidx=fidx
        self.purity=purity
        self.f_type= None
        self.leaf = False
        
    def set_childs(self,lchild,rchild):
        self.lchild=lchild      
        self.rchild=rchild
        
    def isleaf(self):
        return self.leaf
    def isless_than_eq(self, X):
        if X[self.fidx]<=self.split: 
            return True
        else:
            return False
    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.klasslabel,self.purity)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split)

In [5]:
def getSplits(categories):
    categories=set(categories)
    tsplits=t.get_powerset(categories,len(categories)-1) # get all the power sets with the given cardinality...
    flist=[]
    for s in tsplits:
        if not s in flist:
            r=categories.difference(s)
            flist.append(s)
            flist.append(r)
    olist=[]
    for s in flist:
        ilist=[]
        for k in s:
            ilist.append(k)
        olist.append(tuple(ilist))
    return olist

In [6]:
class DecisionTree:
    def __init__(self, purityp, exthreshold, maxdepth=10):        
        self.purity = purityp
        self.exthreshold = exthreshold
        self.maxdepth = maxdepth
        pass
    def test(self, X):
        nexamples, nfeatures=X.shape
        pclasses=self.predict(X)
        return pclasses
    def predict(self, X):
        z=[]
        #print "Numver of rows to preduct ", X.shape[0]
        for idx in range(X.shape[0]):
            z.append(self._predict(self.tree,X[idx]))
        return np.array( z )
    def _predict(self,node, X):
        #print X
      
        temp=self.tree
        while temp!=None:
            if temp.f_type == "categorical":
                #print "cat"
                if np.in1d(X[temp.fidx],temp.split  ):
                    #print "l"
                    temp=temp.lchild
                else:
                    #print "r"
                    temp=temp.rchild
            elif temp.f_type == "numerical":
                #print "num"
                if X[temp.fidx] >temp.split:
                    #print "r"
                    temp=temp.rchild
                else :
                    #print "l"
                    temp=temp.lchild
            else:
                #print "The class i got is", temp.klasslabel
                return temp.klasslabel
    def train(self, X, Y):
        nexamples, nfeatures = X.shape
        # # now go and train a model for each class...
        # YOUR CODE HERE
        #raise NotImplementedError()
        self.tree = self.build_tree(X,Y,0)
        return
    
    def build_tree(self, X, Y, depth):
        nexamples, nfeatures = X.shape 
        #print "BUILD TREE " , nexamples ," number of rows" 
        klasses, sizekclasses =np.unique(Y, return_counts=True);
        #print klasses, sizekclasses
       
        purities = [float(n)/nexamples  for n in sizekclasses]
        #print purities
        
        nodePurity = max (purities)
        klabel = np.argmax(purities)
        
        thisNode = Node(nodePurity)

        if nexamples<=self.exthreshold or nodePurity>= self.purity or self.maxdepth<=depth+1:
            #print "Class assigned is",klasses[klabel]
            thisNode.klasslabel = klasses[klabel]
            thisNode.leaf = True
            return thisNode
        
        split, gain, left, right, fidx, f_type = 0,0,None, None, 0,None
        
        # not specifying feature 14 and not using nfeatures becasue to takes too long
        for f in range(nfeatures):
            if isinstance(X[0,f],long):
                gain1 , split1  = self.evaluate_numerical_attribute(X[:,f],Y) 
                f_type1 = "numerical"
                #print "Numerical fidx" , f , " Gain ", gain1
            else:
                ## TODO: REMOVE COMMENT
                #gain1 , split1 = -1 , []
                gain1 , split1 = self.evaluate_categorical_attribute(X[:,f],Y) 
                f_type1 = "categorical"
                #print "Categorical fidx" , f , " Gain ", gain1
            if gain1 > gain:
                fidx =f
                gain = gain1
                split = split1
                f_type = f_type1
                # yahin par masla hai
                #left = left1
                #right = right1
        thisNode.fidx = fidx
        thisNode.score = gain
        thisNode.split = split
        thisNode.f_type = f_type
        
        # if attribute is categorical then manage differently
        lchild , rchild = None, None
        if f_type == "categorical":
            #print "Best gain at categorical feature " , fidx, " ", split, " ",gain
            mask = np.in1d(X[:,fidx],split  )
            inv_mask = np.invert(mask)
        else:
            #print "Best gain at numerical feature " , fidx, " ", split, " ",gain
            mask = X[: ,fidx] <= split
            inv_mask = np.invert(mask)
        
        #print "Left ", len(X[mask]) , "Right", len(X[inv_mask])
        
        
        #for i in xrange( len(X[mask]) ):
        #    print X[i], " ", Y[i]
        # if there is no left child
        
        if len( X[inv_mask] ) == 0 :
            #print "Nothing for DN",  " So class assigned is ", klasses[klabel]
            thisNode.klasslabel = klasses[klabel]
            thisNode.leaf = True
            thisNode.score = 0
            thisNode.split = None
            thisNode.f_type = None
            
            return thisNode
        else: 
            lchild = self.build_tree( X[mask],Y[mask],depth+1 )    
            rchild = self.build_tree( X[inv_mask],Y[inv_mask],depth+1 )
        
        thisNode.set_childs(lchild,rchild)
        return thisNode
        ''' continue buildtree
        if gain==99:
            thisNode.klasslabel = klasses[klabel]
            thisNode.leaf = True
            return thisNode
        '''
     
        
    def evaluate_categorical_attribute(self,feat, y):
        x =  feat.ravel()
        y = y.ravel()
        # Datastructure
        table = defaultdict( lambda: defaultdict(int) )
        for i in xrange( len(x) ):
            table[ x[i] ][ y[i]] += 1
        
        # Counts
        n = len(x)
        freq = Counter(y)
        #print "Numver of rows receieved ",n
        #print n, freq_sum
        freq_sum = sum( freq.values() )
        unique_classes = freq.keys()
        
        # powersets
        categories = set(x)
        splits = getSplits(categories) if len(categories) > 1 else tuple(categories)
        indexes = range(len(splits))
        even_indices= [i for i in indexes if i %2 == 0] 
        
        max_gain, max_gain_split = 0, []
        # Dataset Entropy
        entropy = 0
        for k,v in freq.iteritems():
            if v > 0:
                p = float( v ) / freq_sum
                entropy+= p*math.log(p,2)
        entropy =  abs( entropy)
        
        #print entropy
        #print "Number of examples in x ", len(x)
        best_dy_ent,  best_dn_ent =0,0
        for i in even_indices:
            dy_entropy =   calulate_entropies_categorical( splits[i], table, n , unique_classes )
            # handling the case when only 1 tuple exists in powerset
            if len( splits ) == 1:
                dn_entropy = 0
            else:
                dn_entropy =  calulate_entropies_categorical( splits[i+1], table,n , unique_classes )

            gain = entropy - (dy_entropy +dn_entropy)
            #print "DATASET ENTROP", entropy  ,"Gain = ", gain , " DY", dy_entropy, " DN", dn_entropy
            #print "=========================================="
            if gain > max_gain:
                max_gain = gain
                max_gain_split = splits[i]
                best_dy_ent = dy_entropy
                best_dn_ent = dn_entropy

        #print "BEST SCORE ", max_gain, " AT SPLIT ", max_gain_split ,"Entropy", entropy ," Dy_ent", best_dy_ent, "dn_ent",best_dn_ent
        return max_gain, max_gain_split
    
    def evaluate_numerical_attribute(self,feat, Y ):
        x =  feat.ravel()
        y = Y.ravel()

        # Counts
        n = len(x)
        freq = Counter(y)
        freq_sum = sum( freq.values() )
        #print "Number of rows ", n
        unique_classes = list(np.unique( Y ))

        # k arg sort
        sorted_indices = np.argsort(feat)
        sorted_feature =  x[ sorted_indices ]
        sorted_labels = y[sorted_indices]  

        # datastructure
        table = np.zeros( [ n ,len(unique_classes)] )
        for i in xrange(len(sorted_feature)):
            if i == 0:
                table[0, unique_classes.index( sorted_labels[i] ) ] = 1
            else:
                prev = table[i-1 ] 
                table[i ] = prev        
                table[i, unique_classes.index( sorted_labels[i] ) ] = prev[ unique_classes.index( sorted_labels[i] ) ] + 1

        # Dataset Entropy
        entropy = 0
        for k,v in freq.iteritems():
            if v > 0:
                p = float( v ) / freq_sum
                entropy+= p*math.log(p,2)
        entropy =  abs( entropy)

        #print freq
        #print "Entropy " , entropy

        max_gain, max_gain_split = 0, 0   
        for i in  xrange( table.shape[0] ):
            dy_entropy =  calulate_entropies_numerical( table[i], n )
            dn = sub_vec( table[i], table[n-1] )
            dn_entropy =  calulate_entropies_numerical( dn , n )
            gain = entropy - (dy_entropy +dn_entropy)
            if gain > max_gain:
                # first check if the split point doesnt comes on later
                
                max_gain = gain
                max_gain_split = sorted_feature[i]

        return max_gain, max_gain_split
    def __str__(self):
        return self.__print(self.tree)        
        
     

    def print_(self):
        self.__print(self.tree)
    def __print(self,node,depth=0):
        ret = ""
        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild,depth+1)
        # Print own value
        ret += "\n" + ("    "*depth) + node.get_str()
        # Print left branch
        if node.lchild:
            ret += self.__print(node.lchild,depth+1)
        return ret    
        
        
    

# Lets start 

In [7]:
col_names = ["age","workclass","fnlwgt","education","education_num","marital_status","occupation","relationship","race","sex","capital_gain","capital_loss","hours_per_weak","native_country","class"]
data = pd.read_csv( "adult.data.txt",names= col_names ,delimiter=',' )

In [8]:
train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])
x_train=np.asarray(train[["age","workclass","fnlwgt","education","education_num","marital_status","occupation","relationship","race","sex","capital_gain","capital_loss","hours_per_weak","native_country"]])
y_train = np.asarray(train[['class']])

In [9]:
print len(x_train)

19536


In [ ]:
%%time
dt=DecisionTree(0.90,20)
dt.train(x_train[:,:13], y_train)

### Decision Tree Structure

In [36]:
print dt


            C(class= >50K,Purity=0.994694960212)
        I(Fidx=10,Score=0.0667370407937,Split=5013)
                C(class= >50K,Purity=0.968)
            I(Fidx=11,Score=0.0465664344087,Split=1740)
                            C(class= <=50K,Purity=0.958333333333)
                        I(Fidx=2,Score=0.285286352008,Split=70387)
                            C(class= >50K,Purity=1.0)
                    I(Fidx=10,Score=0.0185066201989,Split=3103)
                                    C(class= >50K,Purity=0.853801169591)
                                I(Fidx=4,Score=0.0117058058791,Split=14)
                                    C(class= >50K,Purity=0.756332931242)
                            I(Fidx=12,Score=0.0194038432834,Split=35)
                                    C(class= <=50K,Purity=0.560606060606)
                                I(Fidx=0,Score=0.0694256001409,Split=47)
                                    C(class= >50K,Purity=0.705882352941)
                        I(Fidx=0,Score

# Running Test Set

In [12]:
x_test=np.asarray(test[["age","workclass","fnlwgt","education","education_num","marital_status","occupation","relationship","race","sex","capital_gain","capital_loss","hours_per_weak","native_country"]])
y_test = np.asarray(test[['class']])

In [37]:
z = dt.test(x_test[:,:13]  )

In [38]:

acc=  float( sum( z== y_test.ravel() ) ) / len(x_test) * 100

# Accuracy of Test Set

In [39]:
print "Test set accuracy percantage: " , acc

Test set accuracy percantage:  85.1834791955


# Testing hyperparameters

In [ ]:
%%time
# Hyperparameters
n_examples = [ 5,10,15,20 ]
purities = [ 80,85,90,95 ]
depth = [5,10]
acc_list = []
best_nexamples , best_purities, best_depth , best_accuracy = 0,0,0,0
for n in n_examples:
    for p in purities:
        for d in depth:
            dt=DecisionTree(p,n,d)
            dt.train(x_train[:,:13], y_train[:])
            z = dt.test(x_test[:500,:13]  )
            acc=  float( sum( z== y_test[:].ravel() ) ) / len(x_test[:]) * 100
            acc_list.append(acc)
            if acc > best_accuracy:
                best_accuracy = acc
                best_nexamples = n
                best_purities = p
                best_depth = d
            
            

In [ ]:
print "Accuracy ", best_accuracy, " N_Example " , best_nexamples, " Purities " , best_purities , " Best_depth " , best_depth

In [ ]:
print acc_list

# Final Best Run and Confusion matrix

In [32]:
def get_confusion_matrix( actual, predicted ):
    # calculate the confusion matrix; labels is numpy array of classification labels
    confusion_matrix = np.zeros([2,2])
    for a, p in zip(actual, predicted):
        if a == ' <=50K':
            if p == ' <=50K':
                confusion_matrix[0][0] +=1
            else:
                confusion_matrix[0][1] +=1
        else:
            if p == ' <=50K':
                confusion_matrix[1][0] +=1
            else:
                confusion_matrix[1][1] +=1
    return confusion_matrix


In [10]:
dt=DecisionTree(0.90,20)
dt.train(x_train[:,:13], y_train[:])

In [33]:
z = dt.test(x_test[:,:13]  )
acc=  float( sum( z== y_test.ravel() ) ) / len(x_test) * 100
conf_matrix = get_confusion_matrix( y_test.ravel(),z )

In [35]:
print "Accuracy= ", acc
print confusion_matrix

Accuracy=  85.1834791955
[[ 4759.   194.]
 [  771.   789.]]
